In [ ]:
# todo: train generative model on the recovered sentences

In [1]:
# Imports.
import pandas as pd
from enum import Enum
import nltk
from nltk.corpus import wordnet as wn
from spacy.matcher import Matcher
import spacy
import en_core_web_sm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import csv

In [2]:
# Global variables.
nlp = en_core_web_sm.load()
passive_matcher = Matcher(nlp.vocab)
passive_rule = [{'DEP':'nsubjpass'}, {'DEP': 'auxpass'}]
passive_matcher.add('Passive',None,passive_rule)

clause_matcher = Matcher(nlp.vocab)
clause_rule = [{'DEP':'det', 'OP': '*'}, {'DEP':'amod', 'OP': '*'}, 
               {'DEP':'nsubj'}, {'DEP':'ROOT'}, {'DEP':'det', 'OP': '*'}, 
               {'DEP': 'dobj', 'OP': '*'}, {'DEP': 'prep'}, {'DEP': 'pobj'}]
clause_matcher.add('Clause',None,clause_rule)

pp_matcher = Matcher(nlp.vocab)
pp_rule = [{'DEP':'prep'}, {'DEP':'det', 'OP': '*'}, {'DEP':'amod', 'OP': '*'}, {'DEP':'pobj'}]
pp_matcher.add('PrepPhrase',None,pp_rule)

In [3]:
# Read in the corpus for example sentences.
corpus_filename = 'C:\\Users\\Christian\\Downloads\\en-es.txt\\subtitles.en'   

In [4]:
# Class definitions.
class Tense(Enum):
    SIMPLEPRES = 0
    SIMPLEPAST = 1
    PRESPERFECT = 2
    PASTPERFECT = 3
    PRESPROG = 4
    PASTPROG = 5
    PASTPERFPROG = 6
    
    def __str__(self):
        if self == Tense.SIMPLEPRES:
            return 'Simple present'
        if self == Tense.SIMPLEPAST:
            return 'Simple past'
        if self == Tense.PRESPERFECT:
            return 'Present perfect'
        if self == Tense.PASTPERFECT:
            return 'Past perfect'
        if self == Tense.PRESPROG:
            return 'Present progressive'
        if self == Tense.PASTPROG:
            return 'Past progressive'
        if self == Tense.PASTPERFPROG:
            return 'Past perfect progressive'
        
    def __repr__(self):
        return str(self)
    

class Person(Enum):
    I = 0
    WETHEY = 1
    YOU = 2
    HSI = 3
    
    def __str__(self):
        if self == Person.I:
            return 'I'
        if self == Person.WETHEY:
            return 'We | they'
        if self == Person.YOU:
            return 'You'
        if self == Person.HSI:
            return 'He | she | it'
        
    def __repr__(self):
        return str(self)
    

class Verb:
    def __init__(self, df_row):
        self.lemma = df_row['lemma']
        # Organize tenses and person in dictionary with (tense, person) key
        self.forms = {}
        # Simple present for all four persons
        self.forms[(Tense.SIMPLEPRES, Person.I)] = df_row['i_pres']
        self.forms[(Tense.SIMPLEPRES, Person.WETHEY)] = df_row['wethey_pres']
        self.forms[(Tense.SIMPLEPRES, Person.YOU)] = df_row['you_pres']
        self.forms[(Tense.SIMPLEPRES, Person.HSI)] = df_row['hsi_pres']
        # Simple past for all four persons
        self.forms[(Tense.SIMPLEPAST, Person.I)] = df_row['i_past']
        self.forms[(Tense.SIMPLEPAST, Person.WETHEY)] = df_row['wethey_past']
        self.forms[(Tense.SIMPLEPAST, Person.YOU)] = df_row['you_past']
        self.forms[(Tense.SIMPLEPAST, Person.HSI)] = df_row['hsi_past']
        # Other tenses can be formed by the combination of present or past participle with corresponding form of "be"
        # Persent perfect
        self.forms[(Tense.PRESPERFECT, Person.I)] = 'have ' + df_row['pastpart']
        self.forms[(Tense.PRESPERFECT, Person.WETHEY)] = 'have ' + df_row['pastpart']
        self.forms[(Tense.PRESPERFECT, Person.YOU)] = 'have ' + df_row['pastpart']
        self.forms[(Tense.PRESPERFECT, Person.HSI)] = 'has ' + df_row['pastpart']
        # Past perfect
        self.forms[(Tense.PASTPERFECT, Person.I)] = 'had ' + df_row['pastpart']
        self.forms[(Tense.PASTPERFECT, Person.WETHEY)] = 'had ' + df_row['pastpart']
        self.forms[(Tense.PASTPERFECT, Person.YOU)] = 'had ' + df_row['pastpart']
        self.forms[(Tense.PASTPERFECT, Person.HSI)] = 'had ' + df_row['pastpart']
        # Present progressive
        self.forms[(Tense.PRESPROG, Person.I)] = 'am ' + df_row['prespart']
        self.forms[(Tense.PRESPROG, Person.WETHEY)] = 'are ' + df_row['prespart']
        self.forms[(Tense.PRESPROG, Person.YOU)] = 'are ' + df_row['prespart']
        self.forms[(Tense.PRESPROG, Person.HSI)] = 'is ' + df_row['prespart']
        # Past progressive
        self.forms[(Tense.PRESPROG, Person.I)] = 'was ' + df_row['prespart']
        self.forms[(Tense.PRESPROG, Person.WETHEY)] = 'were ' + df_row['prespart']
        self.forms[(Tense.PRESPROG, Person.YOU)] = 'were ' + df_row['prespart']
        self.forms[(Tense.PRESPROG, Person.HSI)] = 'was ' + df_row['prespart']
        # Past perfect progressive
        self.forms[(Tense.PASTPERFPROG, Person.I)] = 'have been ' + df_row['prespart']
        self.forms[(Tense.PASTPERFPROG, Person.WETHEY)] = 'have been ' + df_row['prespart']
        self.forms[(Tense.PASTPERFPROG, Person.YOU)] = 'have been ' + df_row['prespart']
        self.forms[(Tense.PASTPERFPROG, Person.HSI)] = 'has been ' + df_row['prespart']

    def get_form(tense, person):
        ''' Return the correct verb phrase given the tense and the form '''
        return self.forms[tense, person]

In [5]:
# Function definitions.
def get_examples(lemma, verb_set):
    examples = []     
    with open(corpus_filename,'rb') as f:
        line = f.readline()
        cnt = 1
        while line:
            line_formatted = line.strip().decode('utf-8').replace('"', '').replace('-', '')

            if any(i in line_formatted for i in verb_set):
                examples.append(line_formatted.lower())
            line = f.readline()
            cnt += 1
            if cnt % 1000000 == 0:
                print('.', end='')
                    
                        
    return examples

In [6]:
def cosine_sim_vectors(vec1, vec2):
    vec1 = vec1.reshape(1, -1)
    vec2 = vec2.reshape(1, -1)
    
    return cosine_similarity(vec1, vec2)[0][0]

In [7]:
# Practice phrase template structure.
# [det] | [mod] | [nsubj] | [verb] | [det] | [mod] | [dobj] | [prep] | [det] | [mod] | [pobj]

In [8]:
def get_pieces(examples_list, lemma):
    
    pieces = []
    
    lemma_matcher = Matcher(nlp.vocab)
    lemma_rule = [{'DEP':'ROOT', 'LEMMA': lemma, 'POS': 'VERB'}]
    lemma_matcher.add('LemmaInclusive',None,lemma_rule)
    
    for item in examples_list:
        doc = nlp(item)
        
        for sent in doc.sents:
            sent_split = sent.text
            sent_split_doc = nlp(sent_split)
            clause = clause_matcher(sent_split_doc)
            lemma_check = lemma_matcher(sent_split_doc)        
        
            if clause and lemma_check and '?' not in sent_split:
                original = sent_split
                d_np = []
                m_np = []
                h_np = []

                verb = []

                d_vp = []
                m_vp = []
                h_vp = []

                prep = []

                d_pp = []
                m_pp = []
                h_pp = []

                for tok in sent_split_doc:
                    if tok.dep_ == 'nsubj':     
                        d_np = [desc for desc in tok.subtree if (desc.dep_ == "det")]
                        m_np = [desc for desc in tok.subtree if (desc.dep_ == "amod")]    
                        h_np = [tok]

                    elif tok.dep_ == 'ROOT':
                        verb = [tok]

                    elif tok.dep_ == 'dobj':
                        d_vp = [desc for desc in tok.subtree if (desc.dep_ == "det")]
                        m_vp = [desc for desc in tok.subtree if (desc.dep_ == "amod")]
                        h_vp = [tok]

                    else:
                        pass

                p_phrase = pp_matcher(sent_split_doc)
                if p_phrase:
                    for match_id, start, end in p_phrase:
                        string_id = nlp.vocab.strings[match_id]
                        span = doc[start:end]
                        p_doc = nlp(span.text)
                    prep = [[tok for tok in p_doc if (tok.dep_ == "ROOT" or 'prep')][0]]  
                    d_pp = [tok for tok in p_doc if (tok.dep_ == "det")]
                    m_pp = [tok for tok in p_doc if (tok.dep_ == "amod")]
                    h_pp = [tok for tok in p_doc if (tok.dep_ == "pobj")]


                dict_pieces = {

                        'd_np': d_np,
                        'm_np': m_np,
                        'h_np': h_np,
                        'verb': verb,
                        'd_vp': d_vp,
                        'm_vp': m_vp,
                        'h_vp': h_vp,
                        'prep': prep,
                        'd_pp': d_pp,
                        'm_pp': m_pp,
                        'h_pp': h_pp

                }
                
                pieces_cleaned = []
                # This will need to be altered so we can also account for empty objects etc.
                for key, value in dict_pieces.items():
                    if len(value) > 0:
                        if key == 'verb':
                            pieces_cleaned.append(value[0].text.upper())
                        else:
                            pieces_cleaned.append(value[0].text)
                        
                proposed = ' '.join(pieces_cleaned)

                vectorizer = CountVectorizer().fit_transform([original, proposed])
                vectors = vectorizer.toarray()
                csim = cosine_similarity(vectors)
                similarity = cosine_sim_vectors(vectors[0], vectors[1])
                
                if similarity > 0.84:
                    d = {'source': original, 'parsed': proposed, 'similarity': similarity}
                    pieces.append(d)
                    print(original)
                    print(proposed)
                    print('Similarity: {}'.format(similarity))
                    print('\n')
                    
    
    return pieces

In [9]:
def export_examples(examples_list, lemma):
    filedata = 'example_sentences_01.csv'
    fieldnames = ['lemma', 'examples']
    examples = []
    with open(filedata, 'a', newline='', encoding='utf-8') as f1:
        csv_output1 = csv.DictWriter(f1, delimiter=',', fieldnames=fieldnames)
        for item in examples_list:
            examples.append(item['parsed'])
        csv_output1.writerow({'lemma': lemma, 'examples': examples})

In [10]:
# Load the dataframe.
with open('verb_table_02.csv', 'r') as f:
    df = pd.read_csv(f)

In [11]:
# Create the set of verb objects.
verbs = []
for index, row in df.iterrows():
    verbs.append(Verb(row))

In [12]:
# verbs_test = [verbs[343]]

In [13]:
# Test verbs: 50:feed, 343:drop

In [14]:
# Main.
def main():
    for verb in verbs[4:]:     # Change back to 'verbs'
        lemma = verb.lemma
        verb_forms = verb.forms.items()
        verb_set = set()
        for key, value in verb_forms:
            verb_set.add(value)

        print('Examining verb: {}'.format(lemma))
        example_sentences = get_examples(lemma, verb_set)
        print('\tNumber of examples: {}\n'.format(len(example_sentences)))
        print('\nExtracting plausible examples ...\n')
        pieces = get_pieces(example_sentences, lemma)
        pieces_sorted = sorted(pieces, key=lambda k: k['similarity'], reverse=True)
        print('Number of plausible example sentences: {}'.format(len(pieces_sorted)))
        export_examples(pieces_sorted, lemma)
        
        # Write a CSV header manually for now ----- writer.writeheader()
        # remove print statements and add progress bar
        # add statements to confirm export
        # may need to handle errors if some verbs dont have any good tokens

In [16]:
if __name__ == '__main__':
    main()

Examining verb: bear
.............................................................	Number of examples: 53402


Extracting plausible examples ...

believe you me, you bore me to distraction.
you BORE me to distraction
Similarity: 0.9036961141150641


he bore a grudge for something else.
he BORE a grudge for something
Similarity: 0.9128709291752769


these dispatches bear the seal of spain.
these dispatches BEAR the seal of spain
Similarity: 0.9999999999999997


everybody bear witness to that.
everybody BEAR witness to that
Similarity: 0.9999999999999999


you bore me to tears, sarge.
you BORE me to tears
Similarity: 0.9128709291752769


and it bears on it the sign of the house of achmed.
it BEARS the sign of achmed
Similarity: 0.8660254037844388


it bears the name of tadamori.
it BEARS the name of tadamori
Similarity: 1.0000000000000002


you bear that in mind.
you BEAR that in mind
Similarity: 0.9999999999999999


they bear a flag of friendship.
they BEAR a flag of friendship
Similari

then my sister beat me to it.
sister BEAT me to it
Similarity: 0.8451542547285165


the cops beat you to it.
the cops BEAT you to it
Similarity: 1.0000000000000002


 they beat us to it.
they BEAT us to it
Similarity: 0.9999999999999999


we beat them by 6.
we BEAT them by 6
Similarity: 1.0


 but they beat 'em to it.
they BEAT 'em to it
Similarity: 0.9128709291752769


they beat us to it!
they BEAT us to it
Similarity: 0.9999999999999999


who beat all of us!
who BEAT all of us
Similarity: 0.9999999999999999


you beat us to it.  
you BEAT us to it
Similarity: 0.9999999999999999


mark beat him to it.
mark BEAT him to it
Similarity: 0.9999999999999999


they beat him to death with a whip.
they BEAT him with a whip
Similarity: 0.8451542547285165


you stupid mullethead, he beat you with nothing.
he BEAT you with nothing
Similarity: 0.848528137423857


the fungus beat us to it.
the fungus BEAT us to it
Similarity: 1.0000000000000002


 yeah, they beat on me some.  
they BEAT some on me


the tiger beat us to him .
the tiger BEAT us to him
Similarity: 1.0000000000000002


someone beat a girl to death.
someone BEAT a girl to death
Similarity: 0.9999999999999999


my heart beats for rusty.
heart BEATS for rusty
Similarity: 0.8944271909999159


it beat in them until the death.
it BEAT until the death
Similarity: 0.8451542547285165


they beat you like rugs.
they BEAT you like rugs
Similarity: 0.9999999999999999


nobody beats it like this, doc.
nobody BEATS it like this
Similarity: 0.9128709291752769


my heart beats for her
heart BEATS for her
Similarity: 0.8944271909999159


they beat us with chains.
they BEAT us with chains
Similarity: 0.9999999999999999


 we beat indiana on thursday.
we BEAT indiana on thursday
Similarity: 0.9999999999999999


then they beat each other to death.
they BEAT each other to death
Similarity: 0.9258200997725515


 clark beat you to it.
clark BEAT you to it
Similarity: 0.9999999999999999


you beat him with belt.
you BEAT him with belt
Simil

you beat me to it.
you BEAT me to it
Similarity: 0.9999999999999999


i beat him for 40,000.
i BEAT him for 40,000
Similarity: 0.9999999999999999


you beat mel to death.
you BEAT mel to death
Similarity: 0.9999999999999999


razr beat us to it.
razr BEAT us to it
Similarity: 0.9999999999999999


flowers beat her to death.
flowers BEAT her to death
Similarity: 0.9999999999999999


they beat the guy to death.
they BEAT the guy to death
Similarity: 1.0000000000000002


dodgers beat minnesota in seven.
dodgers BEAT minnesota in seven
Similarity: 0.9999999999999999


you beat him into submission.
you BEAT him into submission
Similarity: 0.9999999999999999


fluorescent beat from them.
fluorescent BEAT from them
Similarity: 1.0


he beat him to death with a crowbar.
he BEAT him with a crowbar
Similarity: 0.8451542547285165


sorry,someone beat you to it.
someone BEAT you to it
Similarity: 0.9128709291752769


tuna beat me to it.
tuna BEAT me to it
Similarity: 0.9999999999999999


my heart b

shit, he beat me by 1/10.
he BEAT me by 1/10
Similarity: 0.9128709291752769


you beat him in front of strangers.
you BEAT him of strangers
Similarity: 0.8451542547285165


somebody beat this man to death.
somebody BEAT this man to death
Similarity: 1.0000000000000002


(hearts beating over monitor)
hearts BEATING over monitor
Similarity: 1.0


(heart beating on monitor)
heart BEATING on monitor
Similarity: 1.0


my heart beats for you.
heart BEATS for you
Similarity: 0.8944271909999159


my heart beats for you.
heart BEATS for you
Similarity: 0.8944271909999159


well, agent gaad beat you to it.
gaad BEAT you to it
Similarity: 0.8451542547285165


somebody beat us to it.
somebody BEAT us to it
Similarity: 0.9999999999999999


our hearts beat as o...
hearts BEAT as
Similarity: 0.8660254037844388


you beat us to it.
you BEAT us to it
Similarity: 0.9999999999999999


my father beat us as children.
father BEAT us as children
Similarity: 0.9128709291752769


 alice beat us to it.
alice BE

...

i bought myself till tonight.
i BOUGHT myself till tonight
Similarity: 1.0


we buy a shitload of gasoline.
we BUY a shitload of gasoline
Similarity: 0.9999999999999999


then the dutch bought the planes from england.
the dutch BOUGHT the planes from england
Similarity: 0.9486832980505138


he bought them from you.
he BOUGHT them from you
Similarity: 0.9999999999999999


where pétur bought something on monday.
pétur BOUGHT something on monday
Similarity: 0.9128709291752769


he bought a pack of condoms.
he BOUGHT a pack of condoms
Similarity: 0.9999999999999999


my grandpappy bought this land in 1902.
grandpappy BOUGHT this land in 1902
Similarity: 0.9258200997725515


she bought the ring from me.
she BOUGHT the ring from me
Similarity: 1.0000000000000002


i bought some stuff from downtown.
i BOUGHT some stuff from downtown
Similarity: 0.9999999999999999


you bought a painting from him.
you BOUGHT a painting from him
Similarity: 0.9999999999999999


we'ii buy a ticket from them..
we

i bought them on hsn.
i BOUGHT them on hsn
Similarity: 1.0


and you bought a map of venezuela.
you BOUGHT a map of venezuela
Similarity: 0.9128709291752769


kadam bought this bungalow from me..
kadam BOUGHT this bungalow from me
Similarity: 1.0000000000000002


and i bought lots of beer.
i BOUGHT lots of beer
Similarity: 0.8944271909999159


tonight, richard hammond buys a cup of coffee.
hammond BUYS a cup of coffee
Similarity: 0.8451542547285165


i bought it before i
i BOUGHT it before i
Similarity: 1.0000000000000002


i bought both of them.
i BOUGHT both of them
Similarity: 1.0


i bought into it.
i BOUGHT into it
Similarity: 1.0000000000000002


chen bought a mojito at 11:34 p.m.
chen BOUGHT a mojito at 11:34
Similarity: 1.0000000000000002


i bought tea from him yesterday.
i BOUGHT tea from him
Similarity: 0.8944271909999159


i bought something with it.
i BOUGHT something with it
Similarity: 1.0


mom bought that for both of us!
mom BOUGHT that of us
Similarity: 0.845154254728

 i bought them with math.
i BOUGHT them with math
Similarity: 1.0


we bought in 2007.
we BOUGHT in 2007
Similarity: 1.0


if you bought at 150...
you BOUGHT at 150
Similarity: 0.8944271909999159


i bought this home with money i earned
i BOUGHT this home with money
Similarity: 0.9128709291752769


your far rockaway facility bought a load of fuel...
far facility BOUGHT a load of fuel
Similarity: 0.8660254037844387


i buy a packet from time to time.
i BUY a packet to time
Similarity: 0.8838834764831843


i bought it in egypt.
i BOUGHT it in egypt
Similarity: 1.0


in egypt, i bought the gun from one.
i BOUGHT the gun from one
Similarity: 0.8451542547285165


i bought it at whsmith.
i BOUGHT it at whsmith
Similarity: 1.0


i bought a banjolele on craigslist.
i BOUGHT a banjolele on craigslist
Similarity: 1.0


i bought a condo in florida.
i BOUGHT a condo in florida
Similarity: 1.0


we buy food in bulk.
we BUY food in bulk
Similarity: 0.9999999999999999


and then they buy a lot of dri

we bought it through friends.
we BOUGHT it through friends
Similarity: 0.9999999999999999


i bought this for wendi.
i BOUGHT this for wendi
Similarity: 1.0


well... a trillion dollars buys a lot of things.
dollars BUYS a lot of things
Similarity: 0.8451542547285165


everyone buys from us.
everyone BUYS from us
Similarity: 1.0


he bought it for all of you.
he BOUGHT it for all
Similarity: 0.8451542547285165


i bought the building from larry yesterday.
i BOUGHT the building from larry
Similarity: 0.9128709291752769


i bought it from you.
i BOUGHT it from you
Similarity: 1.0


i bought the building in foreclosure.
i BOUGHT the building in foreclosure
Similarity: 0.9999999999999999


no, darling, we bought him on sale.
we BOUGHT him on sale
Similarity: 0.8451542547285165


i bought stock in marvel.
i BOUGHT stock in marvel
Similarity: 1.0


my dad bought a lot of stuff.
dad BOUGHT a lot of stuff
Similarity: 0.9128709291752769


i bought it for 300
i BOUGHT it for 300
Similarity: 1.0


KeyboardInterrupt: 